In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
loader = DirectoryLoader(
    path="data",
    glob="info3.txt",       
    loader_cls=TextLoader   
)

In [16]:
docs = loader.load()

In [17]:
docs

[Document(metadata={'source': 'data/info3.txt'}, page_content='India’s economy in 2025 stands as one of the fastest growing in the world, projected at approximately $4.2 trillion in nominal GDP and maintaining an annual growth rate of 6.3% to 6.5%, making it the fourth largest globally in nominal terms and the third largest in purchasing power parity. This momentum is driven by robust domestic consumption, rising foreign direct investment, government-backed infrastructure programs, and the increasing digitization of both urban and rural sectors. Major initiatives such as the National Infrastructure Pipeline and the PM Gati Shakti masterplan are catalyzing manufacturing growth, logistics optimization, and multimodal connectivity across the country, while the Union Budget 2025 has introduced sweeping tax changes, including zero income tax up to ₹12 lakh (₹12.75 lakh with standard deduction), increased TDS thresholds, and longer filing windows to enhance compliance ease. Alongside these d

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 50
)

In [19]:
new_docs = text_splitter.split_documents(documents=docs)

In [20]:
doc_strings = [doc.page_content for doc in new_docs]

In [21]:
db = Chroma.from_documents(new_docs, embedding=embeddings) 
retriever = db.as_retriever(search_kwargs={"k":3})

In [22]:
query = "what is Indian economy?"
docs = retriever.get_relevant_documents(query)
print(docs[0].metadata)
print(docs[0].page_content)

for doc in docs:
    print(doc)

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_36768/2809331465.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


{'source': 'data/info3.txt'}
India’s position as the fastest-growing major economy, even amidst global economic headwinds,
page_content='India’s position as the fastest-growing major economy, even amidst global economic headwinds,' metadata={'source': 'data/info3.txt'}
page_content='India’s economy in 2025 stands as one of the fastest growing in the world, projected at' metadata={'source': 'data/info3.txt'}
page_content='and enhancing India’s global investment appeal, positioning the Indian economy not only as' metadata={'source': 'data/info3.txt'}


In [23]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langchain.prompts import PromptTemplate